In [7]:
from qiime2 import Visualization
import os
import pandas as pd

# Data directory 
data_dir = 'Alien_data'
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

In [2]:
# Multiple Alignment

! qiime alignment mafft \
    --i-sequences $data_dir/rep-seqs-filtered.qza \
    --o-alignment $data_dir/aligned-seqs.qza

Saved FeatureData[AlignedSequence] to: Alien_data/aligned-seqs.qza


In [3]:
# Alignment Masking (removing regions w/ ambiguous aligments)

! qiime alignment mask \
    --i-alignment $data_dir/aligned-seqs.qza \
    --o-masked-alignment $data_dir/masked-aligned-seqs.qza

Saved FeatureData[AlignedSequence] to: Alien_data/masked-aligned-seqs.qza


In [4]:
# DE NOVO Tree Construction

! qiime phylogeny fasttree \
    --i-alignment $data_dir/masked-aligned-seqs.qza \
    --o-tree $data_dir/fasttree.qza

! qiime phylogeny midpoint-root \
    --i-tree $data_dir/fasttree.qza \
    --o-rooted-tree $data_dir/fasttree-rooted.qza

Saved Phylogeny[Unrooted] to: Alien_data/fasttree.qza
Saved Phylogeny[Rooted] to: Alien_data/fasttree-rooted.qza


In [6]:
# DE NOVO Tree Visualisation 

! qiime empress tree-plot \
    --i-tree $data_dir/fasttree-rooted.qza \
    --m-feature-metadata-file $data_dir/taxonomy.qza \
    --o-visualization $data_dir/fasttree-rooted.qzv

Saved Visualization to: Alien_data/fasttree-rooted.qzv


In [12]:
Visualization.load(f'{data_dir}/fasttree-rooted.qzv')

<visualization: Visualization uuid: 42bd6cbe-ff8e-4e4a-abf4-ceb6174ff7e5>

In [3]:
# BOOTSRAPPING 

! qiime phylogeny raxml-rapid-bootstrap \
    --i-alignment $data_dir/masked-aligned-seqs.qza \
    --p-seed 1723 \
    --p-rapid-bootstrap-seed 9384 \
    --p-bootstrap-replicates 100 \
    --p-substitution-model GTRCAT \
    --p-n-threads 3 \
    --o-tree $data_dir/raxml-cat-bootstrap-tree.qza

Saved Phylogeny[Unrooted] to: Alien_data/raxml-cat-bootstrap-tree.qza


In [8]:
# Get reference database (Greengenes)
! wget -nv -O $data_dir/sepp-refs-gg-13-8.qza https://data.qiime2.org/2021.4/common/sepp-refs-gg-13-8.qza

2022-10-29 11:33:49 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/2021.4/common/sepp-refs-gg-13-8.qza [50161069/50161069] -> "Alien_data/sepp-refs-gg-13-8.qza" [1]


In [9]:
# (slow step)
# FRAG IN Construct Tree

! qiime fragment-insertion sepp \
    --i-representative-sequences $data_dir/rep-seqs-filtered.qza \
    --i-reference-database $data_dir/sepp-refs-gg-13-8.qza \
    --p-threads 2 \
    --o-tree $data_dir/sepp-tree.qza \
    --o-placements $data_dir/sepp-tree-placements.qza

Saved Phylogeny[Rooted] to: Alien_data/sepp-tree.qza
Saved Placements to: Alien_data/sepp-tree-placements.qza


In [10]:
# FRAG IN Visualize Tree
! qiime empress tree-plot \
    --i-tree $data_dir/sepp-tree.qza \
    --m-feature-metadata-file $data_dir/taxonomy.qza \
    --o-visualization $data_dir/sepp-tree.qzv

Saved Visualization to: Alien_data/sepp-tree.qzv


In [11]:
Visualization.load(f'{data_dir}/sepp-tree.qzv')

<visualization: Visualization uuid: 68b4ef1b-331c-4899-943e-d43c83848a95>